In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score
import os

###########################
# Checking if GPU is used
###########################

use_cuda=torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()
device=torch.device("cuda:0" if use_cuda else "mps" if use_mps else "cpu")
device

# Data loading and splitting
transform = transforms.Compose([
            transforms.Resize((224, 224)),
            # transforms.RandomRotation(degrees=15),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.4762, 0.3054, 0.2368], [0.3345, 0.2407, 0.2164])
        ])

dataset = "Real_images_processed"
train_root_dir=os.path.join(dataset, 'train')   
val_root_dir= os.path.join(dataset, 'validation') 
test_root_dir= os.path.join(dataset, 'test')


train_data = datasets.ImageFolder(train_root_dir, transform=transform)
test_data = datasets.ImageFolder(test_root_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Train the model
import timm
test_model_path = "fl_checkpointsswintransformer_e5_c5_r5/fl_densenet_5.pth"
print('Best model path:{}'.format(test_model_path))
best_model= timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=2)

checkpoint=torch.load(test_model_path,map_location=device)   # loading best model
best_model.load_state_dict(checkpoint['state_dict'])
best_model.to(device)
best_model.eval()

optimizer = optim.Adam(best_model.parameters())
criterion = nn.CrossEntropyLoss()

def train(model, train_loader, optimizer, criterion, epochs=1):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

train(best_model, train_loader, optimizer, criterion, epochs=1)

# Membership Inference Attack
# Assume we have access to the training set and some test set
# We create a shadow model to simulate the attacker's perspective

class MembershipInferenceAttack:
    def __init__(self, model, train_data, test_data):
        self.model = model
        self.train_data = train_data
        self.test_data = test_data
    
    def attack(self):
        self.model.eval()
        train_membership = []
        test_membership = []

        # Training data inference
        with torch.no_grad():
            for data, target in DataLoader(self.train_data, batch_size=64):
                data = data.to(device)
                target = target.to(device)
                output = self.model(data)
                pred = torch.argmax(output, dim=1)
                correct = pred.eq(target).sum().item()
                train_membership.extend([1] * correct + [0] * (len(data) - correct))
        
        # Test data inference
        with torch.no_grad():
            for data, target in DataLoader(self.test_data, batch_size=64):
                data = data.to(device)
                target = target.to(device)
                output = self.model(data)
                pred = torch.argmax(output, dim=1)
                correct = pred.eq(target).sum().item()
                test_membership.extend([0] * len(data))  # Test data is assumed to be non-members
        
        return train_membership, test_membership

# Instantiate and run the attack
attack = MembershipInferenceAttack(best_model, train_data, test_data)
train_membership, test_membership = attack.attack()

# Combine results and calculate AUC
membership = train_membership + test_membership
labels = [1] * len(train_membership) + [0] * len(test_membership)
auc = roc_auc_score(labels, membership)
print(f'Membership Inference Attack AUC: {auc:.4f}')

Best model path:fl_checkpointsswintransformer_e5_c5_r5/fl_densenet_5.pth
Membership Inference Attack AUC: 0.7704


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score
import os

###########################
# Checking if GPU is used
###########################

use_cuda=torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()
device=torch.device("cuda:0" if use_cuda else "mps" if use_mps else "cpu")
device

# Data loading and splitting
transform = transforms.Compose([
            transforms.Resize((224, 224)),
            # transforms.RandomRotation(degrees=15),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.4762, 0.3054, 0.2368], [0.3345, 0.2407, 0.2164])
        ])

dataset = "Real_images_processed"
train_root_dir=os.path.join(dataset, 'train')   
val_root_dir= os.path.join(dataset, 'validation') 
test_root_dir= os.path.join(dataset, 'test')


train_data = datasets.ImageFolder(train_root_dir, transform=transform)
test_data = datasets.ImageFolder(test_root_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Train the model
import timm
test_model_path = "fl_checkpointsswintransformer_e5_c5_r5/fl_densenet_5.pth"
print('Best model path:{}'.format(test_model_path))
best_model= timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=2)

checkpoint=torch.load(test_model_path,map_location=device)   # loading best model
best_model.load_state_dict(checkpoint['state_dict'])
best_model.to(device)
best_model.eval()

optimizer = optim.Adam(best_model.parameters())
criterion = nn.CrossEntropyLoss()

def train(model, train_loader, optimizer, criterion, epochs=1):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

train(best_model, train_loader, optimizer, criterion, epochs=1)

# Membership Inference Attack
# Assume we have access to the training set and some test set
# We create a shadow model to simulate the attacker's perspective

class MembershipInferenceAttack:
    def __init__(self, model, train_data, test_data):
        self.model = model
        self.train_data = train_data
        self.test_data = test_data
    
    def attack(self):
        self.model.eval()
        train_membership = []
        test_membership = []

        # Training data inference
        with torch.no_grad():
            for data, target in DataLoader(self.train_data, batch_size=64):
                data = data.to(device)
                target = target.to(device)
                output = self.model(data)
                pred = torch.argmax(output, dim=1)
                correct = pred.eq(target).sum().item()
                train_membership.extend([1] * correct + [0] * (len(data) - correct))
        
        # Test data inference
        with torch.no_grad():
            for data, target in DataLoader(self.test_data, batch_size=64):
                data = data.to(device)
                target = target.to(device)
                output = self.model(data)
                pred = torch.argmax(output, dim=1)
                correct = pred.eq(target).sum().item()
                test_membership.extend([0] * len(data))  # Test data is assumed to be non-members
        
        return train_membership, test_membership

# Instantiate and run the attack
attack = MembershipInferenceAttack(best_model, train_data, test_data)
train_membership, test_membership = attack.attack()

# Combine results and calculate AUC
membership = train_membership + test_membership
labels = [1] * len(train_membership) + [0] * len(test_membership)
auc = roc_auc_score(labels, membership)
print(f'Membership Inference Attack AUC: {auc:.4f}')

Best model path:fine_grained_best_model.pth


KeyError: 'model_state_dict'

In [10]:
checkpoint

OrderedDict([('patch_embed.proj.weight',
              tensor([[[[ 0.0343,  0.0644, -0.0269, -0.0406],
                        [ 0.0124,  0.0624,  0.0163,  0.0439],
                        [-0.0264, -0.0106,  0.0152,  0.0066],
                        [-0.0606, -0.0626, -0.0772,  0.0048]],
              
                       [[-0.0027,  0.0321, -0.0685, -0.0798],
                        [ 0.0419,  0.0962,  0.0097,  0.0427],
                        [-0.0165, -0.0004,  0.0169, -0.0054],
                        [-0.0492, -0.0277, -0.0218,  0.0357]],
              
                       [[-0.0384, -0.0231, -0.0661, -0.0670],
                        [-0.0106,  0.0481,  0.0277,  0.0462],
                        [-0.0070, -0.0134,  0.0303,  0.0104],
                        [-0.0185,  0.0042,  0.0186,  0.0370]]],
              
              
                      [[[-0.1078,  0.0545,  0.0786,  0.1162],
                        [-0.0419,  0.0106, -0.0493, -0.0391],
                        [-0